In [1]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch
import os
import numpy as np
import duckdb
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Conectar (ou criar) um banco de dados
con = duckdb.connect(database='jus.duckdb', read_only=False)

In [13]:
con.execute("CREATE TABLE metadadosPublicacao202202 AS SELECT * FROM read_json_auto('./data/202202-stj/metadadosPublicacao202202.json');")

In [14]:
con.execute("DELETE FROM metadadosPublicacao202202 WHERE teor NOT IN ('Concedendo', 'Negando') or teor IS NULL;")

In [3]:
model_name = "Luciano/xlm-roberta-large-finetuned-lener-br"
model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=512, chunk_overlap=256
)

2023-11-12 19:21:02.024516: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [39]:
text = """Acórdão de 2018 – Processo n. 1272/14T8SNT.LS1 – Secção.” 3Inequivocamente, inexistiu qualquer alteração ou mudança da configuração da relação jurídica entre as Partes Processo n. 251/11T8CSC.L S1 - 4 Secção Acordam na Secção Social do Supremo Tribunal de Justiça: AA intentou ação declarativa de condenação, sob a forma de processo comum contra COFAC – Cooperativa de Formação e Animação Cultural, CRL, pedindo se condene a R. a: "a) Reconhecer a existência de um contrato de trabalho entre o Autor e a Ré desde 01/10/1999; b)"""

inputs = tokenizer(text, return_tensors="pt")
inputs.input_ids.shape

torch.Size([1, 147])

In [40]:
outputs = model(**inputs)
predictions = torch.argmax(outputs.logits, dim=2)
outputs.logits.shape

torch.Size([1, 147, 13])

In [41]:
np.array(predictions[0])

array([ 0,  0,  0,  0,  0,  5,  0, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 11, 12, 12, 12,
       12, 12, 12, 12, 12,  0, 12, 12, 12,  0,  0,  0,  0,  0,  0,  0,  1,
        1,  2,  2,  2,  2,  2,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  0,  1,  2,  2,  2,  2,
        2,  2,  2,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  5,  5,  5,  0,  0,  0,  0])

In [4]:
def encontrar_grupos(lista):
    grupos = []
    inicio = None
    fim = None
    for i, valor in enumerate(lista):
        if valor == 11:
            inicio = i
        elif valor == 12 and inicio is not None:
            fim = i
        elif valor != 12 and inicio is not None and fim is not None:
            grupos.append((inicio, fim))
            inicio = None
            fim = None

    return grupos

In [43]:
grupos = encontrar_grupos(np.array(predictions[0]))
grupos

[(7, 19), (47, 55)]

In [45]:
for grupo in grupos:
  print(tokenizer.decode(inputs.input_ids[0][grupo[0]:grupo[1]]))

Processo n. 1272/14T8SNT.LS
Processo n. 251/11T8C


In [8]:
model.config.id2label

{0: 'O',
 1: 'B-ORGANIZACAO',
 2: 'I-ORGANIZACAO',
 3: 'B-PESSOA',
 4: 'I-PESSOA',
 5: 'B-TEMPO',
 6: 'I-TEMPO',
 7: 'B-LOCAL',
 8: 'I-LOCAL',
 9: 'B-LEGISLACAO',
 10: 'I-LEGISLACAO',
 11: 'B-JURISPRUDENCIA',
 12: 'I-JURISPRUDENCIA'}

In [9]:
entities = [model.config.id2label[prediction] for prediction in predictions[0].tolist()]
print(list(zip(text.split(), entities)))

[('Processo', 'O'), ('n.', 'B-JURISPRUDENCIA'), ('251/11T8CSC.L', 'I-JURISPRUDENCIA'), ('S1', 'I-JURISPRUDENCIA'), ('-', 'I-JURISPRUDENCIA'), ('4', 'I-JURISPRUDENCIA'), ('Secção', 'I-JURISPRUDENCIA'), ('Acordam', 'I-JURISPRUDENCIA'), ('na', 'I-JURISPRUDENCIA'), ('Secção', 'I-JURISPRUDENCIA'), ('Social', 'I-JURISPRUDENCIA'), ('do', 'I-JURISPRUDENCIA'), ('Supremo', 'I-JURISPRUDENCIA'), ('Tribunal', 'I-JURISPRUDENCIA'), ('de', 'O'), ('Justiça:', 'O'), ('AA', 'O'), ('intentou', 'O'), ('ação', 'O'), ('declarativa', 'O'), ('de', 'O'), ('condenação,', 'B-ORGANIZACAO'), ('sob', 'B-ORGANIZACAO'), ('a', 'I-ORGANIZACAO'), ('forma', 'I-ORGANIZACAO'), ('de', 'I-ORGANIZACAO'), ('processo', 'I-ORGANIZACAO'), ('comum', 'I-ORGANIZACAO'), ('contra', 'I-ORGANIZACAO'), ('COFAC', 'O'), ('–', 'O'), ('Cooperativa', 'O'), ('de', 'O'), ('Formação', 'O'), ('e', 'O'), ('Animação', 'O'), ('Cultural,', 'O'), ('CRL,', 'O'), ('pedindo', 'O'), ('se', 'O'), ('condene', 'O'), ('a', 'O'), ('R.', 'O'), ('a:', 'O'), ('"a)

In [5]:
data = con.execute("SELECT seqDocumento FROM metadadosPublicacao202202").fetchnumpy()
data["seqDocumento"]

array([144948780, 145620297, 139579332, ..., 142924179, 145440916,
       144826578])

In [6]:
data["seqDocumento"].size

35404

In [7]:
file_paths = []  # para armazenar os caminhos completos dos arquivos
file_names = []  # para armazenar apenas os nomes dos arquivos

# caminhando através do diretório raiz e todos os seus subdiretórios
for dirpath, dirnames, filenames in os.walk('./data/202202-stj/', topdown=True):
    for filename in filenames:
        if filename.endswith('.txt'):
            full_path = os.path.join(dirpath, filename)
            file_paths.append(full_path)
            file_names.append(int(filename.split('.')[0]))

In [15]:
import difflib

def merge_strings(s1, s2, threshold=0.7):
    if len(s2) == 0:
        return s1
    
    # Inicializa o SequenceMatcher
    s = difflib.SequenceMatcher(None, s1, s2)

    # Encontra o bloco de correspondência mais longo
    match = s.find_longest_match(0, len(s1), 0, len(s2))
    # Checa se a correspondência é significativa
    if match.size / len(s2) < threshold:
        return s1 + " | " + s2

    # Constrói a nova string usando a parte correspondente de s1 e o restante de s2
    return s1[:match.a] + s2[match.b:]

In [30]:
jurisprudencias = ['g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 't no REsp 1.591.422/', 't no REsp 1.522.353/', 't no REsp 1.952.026/', 't no REsp 1.442.008/', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'JE 0807836-20.2018.4.05.8', 'JE 0801209-60.2014.4.05.', 'mula 473 do S', 'mula 473 do S', 'RE no 136.236-', 'J 146', 'RE no 136.236-', 'Esp 328', 'Esp 403', 'são no 1.020/2000-TCU-Plen', 'são no 1.020/2000-TCU-Plen', 'MS 24.859-', 'g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'MS 29.025/', 'g no RE 1.281.817/', 'RE 817.338/', 't no REsp 1.522.353/', 't no REsp 1.952.026/', 't no REsp 1.442.008/', 'Processo Administrativo Disciplinar No 23076.000137/2018', 'do de Segurança No 20.16', 'CU no 3088', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'JE 0807836-20.2018.4.05.8', 'JE 0801209-60.2014.4.05.', 't no REsp 1.551.126/', 't no AREsp 1.631.348/', 'mula 473 do Supremo Tribunal', 'mula 473 do Supremo Tribunal', 'dão TCU 2.161', 'g no REsp 1.551.065/', 'g no REsp 1.499.126/', 'cl nos EDcl no AgRg no REsp 1.285.268/', 'g no REsp 1.563.235/', 'g no REsp 1.563.235/', 'g no REsp. 1.314.843/', 'g no REsp. 1.257.473/', 'g no REsp. 1.166.120/', 'g no Ag.1.116.887/', 'g no Ag 1.342.657/', 'g no Ag 1.297.588/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'do de Segurança 28.2', 'do de Segurança 28.2', 'MS 28.2', 'A 200910000001', 'MS 29.025/', 'g no RE 1.281.817/', 'RE 817.338/', 't no REsp 1.442.008/', 't no REsp 1.952.026/', 'MULA 7/ST', 't nos EDcl na AR 6.055/', 'Súmula 7 do ST', 's 7/ST', 't no AREsp 1.163.957/', 't no REsp 1.522.353/', 't no RMS 44.511/', 't no RMS 44.511/', 'g no REsp 1400398/', 'g nos EDcl no AgRg no AREsp. 498.224/', 't no REsp 1344578/', 't no REsp 1.667.120/']

['g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'RE 636.', 't no AREsp 1.631.348/', 't no REsp 1.591.422/', 't no REsp 1.591.422/', 't no REsp 1.522.353/', 't no REsp 1.952.026/', 't no REsp 1.442.008/', 'RE 636.', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'JE 0801209-60.2014.4.05.', 'JE 0801209-60.2014.4.05.', 'mula 473 do S', 'mula 473 do S', 'RE no 136.236-', 'J 146', 'RE no 136.236-', 'J 146', 'Esp 328', 'Esp 403', 'Esp 328', 'Esp 403', 'são no 1.020/2000-TCU-Plen', 'são no 1.020/2000-TCU-Plen', 'MS 24.859-', 'g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'g no REsp 1.563.235/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'MS 29.025/', 'g no RE 1.281.817/', 'RE 817.338/', 't no REsp 1.522.353/', 't no REsp 1.952.026/', 't no REsp 1.442.008/', 'Processo Administrativo Disciplinar No 23076.000137/2018', 'PE no 1.133.0', 'Processo Administrativo Disciplinar No 23076.000137/2018', 'do de Segurança No 20.16', 'CU no 3088', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'RE 636.', 'JE 0807836-20.2018.4.05.8', 'JE 0801209-60.', 'JE 0801209-60.2014.4.05.', 't no REsp 1.551.126/', 't no REsp 1.551.126/', 't no AREsp 1.631.348/', 'mula 473 do Supremo Tribunal', 't no AREsp 1.631.348/', 'mula 473 do Supremo Tribunal', 'mula 473 do Supremo', 'mula 473 do Supremo Tribunal', 'dão TCU 2.161', 'g no REsp 1.551.065/', 'g no REsp 1.499.126/', 'cl nos EDcl no AgRg no REsp 1.285.268/', 'g no REsp 1.563.235/', 'g no REsp 1.563.235/', 'g no REsp. 1.314.843/', 'g no REsp. 1.257.473/', 'g no REsp. 1.166.120/', 'g no Ag.1.116.887/', 'g no Ag 1.342.657/', 'g no Ag 1.297.588/', 'g no REsp 1.270.252/', 'RE 636.', 't no AREsp 1.631.348/', 'do de Segurança 28.2', 'do de Segurança 28.2', 'MS 28.2', 'A 200910000001', 'MS 29.025/', 'MS 29.025/', 'g no RE 1.281.817/', 'aria no 1.104-GM', 'RE 817.338/', 't no REsp 1.442.008/', 't no REsp 1.442.008/', 't no REsp 1.952.026/', 'MULA 7/ST', 't no REsp 1.952.026/', 'MULA 7/ST', 't nos EDcl na AR 6.055/', 't nos EDcl na AR 6.055/', 'mula 7 do ST', 's 7/ST', 't no AREsp 1.163.957/', 't no REsp 1.522.353/', 't no REsp 1.522.353/', 't no RMS 44.511/', 't no RMS 44.511/', 'g no REsp 1400398/', 'g nos EDcl no AgRg no AREsp. 498.224/', 't no REsp 1344578/', 't no REsp 1.667.120/', 't no REsp 1.667.120/']

result = jurisprudencias[0]

for jurisprudencia in jurisprudencias:
  result = merge_strings(result,jurisprudencia)

result

'g no REsp 1.563.235/ | g no REsp. 1.314.843/ | g no REsp. 1.257.473/ | g no REsp. 1.166.120/ | g no Ag.1.116.887/ | g no Ag 1.342.657/ | g no Ag 1.297.588/ | g no REsp 1.270.252/ | RE 636. | t no AREsp 1.631.348/ | do de Segurança 28.2 | A 200910000001 | MS 29.025/ | g no RE 1.281.817/ | RE 817.338/ | t no REsp 1.442.008/ | t no REsp 1.952.026/ | MULA 7/ST | t no AREsp 1.163.957/ | t no REsp 1.522.353/ | t no RMS 44.511/ | g no REsp 1400398/ | g nos EDcl no AgRg no AREsp. 498.224/ | t no REsp 1344578/ | t no REsp 1.667.120/'

In [23]:
def process_file(file_path,file_id):
    with open(file_path, 'r') as f:
        content = f.read()
        texts = text_splitter.split_text(content)
        jurisprudencias = []
        print(file_id,"texts",len(texts))
        for i,text in enumerate(texts):
            #print(file_id,"text",i)
            inputs = tokenizer(text, return_tensors="pt")
            outputs = model(**inputs)
            predictions = torch.argmax(outputs.logits, dim=2)
            grupos = encontrar_grupos(np.array(predictions[0]))
            #print(file_id,"jurisprudencia",len(grupos))
            for grupo in grupos:
                jurisprudencia = tokenizer.decode(inputs.input_ids[0][grupo[0]:grupo[1]])
                jurisprudencias.append(jurisprudencia)
                #print(file_id,jurisprudencia)
        
        if 0 < len(jurisprudencias):
            result = jurisprudencias[0]
            for jurisprudencia in jurisprudencias:
                result = merge_strings(result,jurisprudencia)

            print(result)

In [10]:
file_paths[file_names.index(data["seqDocumento"][0])]

'./data/202202-stj/20220210/144948780.txt'

In [25]:
for index,doc in enumerate(data["seqDocumento"]):
  if doc in file_names:
    process_file(file_paths[file_names.index(doc)],doc)  

144948780 texts 56
g no REsp 1.563.235/ | g no REsp. 1.314.843/ | g no REsp. 1.257.473/ | g no REsp. 1.166.120/ | g no Ag.1.116.887/ | g no Ag 1.342.657/ | g no Ag 1.297.588/ | g no REsp 1.270.252/ | RE 636. | t no AREsp 1.631.348/ | do de Segurança 28.2 | A 200910000001 | MS 29.025/ | g no RE 1.281.817/ | aria no 1.104-GM | RE 817.338/ | t no REsp 1.442.008/ | t no REsp 1.952.026/ | MULA 7/ST | t no AREsp 1.163.957/ | t no REsp 1.522.353/ | t no RMS 44.511/ | g no REsp 1400398/ | g nos EDcl no AgRg no AREsp. 498.224/ | t no REsp 1344578/ | t no REsp 1.667.120/
145620297 texts 10
as Corpus n. 0008126-31.2021.8.27.2700/ | s n. 5008592-97.2013.8.27.2706/ | s n. 5008800-81.2013.827.27
139579332 texts 19
MULA N. 691/ST | mula n. 691/ST | g no HC 623.545/
141816757 texts 32
HC 107.194/ | . 2067076-27.2020.12005 | g no RHC 139.242/ | s no 1500369-67.2021.8.26.04
144148942 texts 19
RE 603.616/ | HC 620.515/ | HC 612.579/ | HC 442.363/ | g no REsp 1.704.746/ | HC 598.051/ | ação penal no 15001

In [10]:
con.close()